# 🤖 AI SEO Architects - Полная демонстрация 14 агентов

## 📋 Описание проекта

**AI SEO Architects** - enterprise-ready мультиагентная RAG-система для автоматизации SEO-агентства с полной архитектурой из 14 специализированных AI-агентов.

### 🏗️ Архитектура системы:

#### 🏢 **Executive Level (2 агента):**
- **Chief SEO Strategist** - стратегическое SEO планирование
- **Business Development Director** - enterprise партнерства

#### 📋 **Management Level (4 агента):**
- **Task Coordination Agent** - LangGraph оркестрация
- **Sales Operations Manager** - pipeline управление
- **Technical SEO Operations Manager** - технические SEO операции
- **Client Success Manager** - управление клиентским успехом

#### ⚙️ **Operational Level (8 агентов):**
- **Lead Qualification Agent** - BANT/MEDDIC квалификация
- **Proposal Generation Agent** - генерация предложений
- **Sales Conversation Agent** - СПИН методология
- **Technical SEO Auditor** - технический аудит
- **Content Strategy Agent** - контентная стратегия
- **Link Building Agent** - линкбилдинг
- **Competitive Analysis Agent** - конкурентный анализ
- **Reporting Agent** - отчетность и аналитика

---

## 🎯 Цель демонстрации

Показать полный цикл работы SEO-агентства от получения лида до создания отчета через взаимодействие всех 14 агентов с реальными LLM (OpenAI GPT-4) и специализированными промптами.

## 📦 Установка зависимостей

Устанавливаем все необходимые библиотеки для работы системы.

### 🔑 Важно для Google Colab:
**Перед запуском настройте секрет OpenAI API:**
1. Нажмите 🔑 (ключ) в левом меню Colab
2. Добавьте секрет: `OPENAI_API_KEY` = ваш OpenAI API ключ
3. Система автоматически получит ключ из секретов

In [ ]:
# Установка зависимостей
!pip install -q openai==1.54.3
!pip install -q langchain==0.2.16
!pip install -q langchain-openai==0.1.25
!pip install -q pydantic==2.9.2
!pip install -q faiss-cpu==1.8.0
!pip install -q python-dotenv==1.0.1
!pip install -q asyncio
!pip install -q nest-asyncio

print("✅ Все зависимости установлены!")

## 🔧 Настройка окружения

Настраиваем переменные окружения и базовую конфигурацию.

In [ ]:
import os
import sys
import asyncio
import nest_asyncio
from typing import Dict, Any, List, Optional
import json
from datetime import datetime
import logging

# Включаем поддержку async в Jupyter
nest_asyncio.apply()

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Функция для настройки API ключа
def setup_openai_key(api_key: str = None):
    """
    Настройка OpenAI API ключа
    
    Args:
        api_key (str): OpenAI API ключ. Если не указан, пытается получить из Colab secrets
    """
    if api_key is None:
        # Пробуем получить из Google Colab secrets
        try:
            from google.colab import userdata
            api_key = userdata.get('OPENAI_API_KEY')
            print("✅ OpenAI API ключ получен из секретов Colab")
        except:
            # Если не в Colab, запрашиваем у пользователя
            try:
                import getpass
                api_key = getpass.getpass("Введите ваш OpenAI API ключ: ")
            except:
                api_key = input("Введите ваш OpenAI API ключ: ")
    
    os.environ["OPENAI_API_KEY"] = api_key
    print("✅ OpenAI API ключ настроен")
    return api_key

# Настройка ключа (будет запрошен при первом запуске агента)
print("🔑 OpenAI API ключ будет получен из секретов Colab или запрошен у пользователя")

## 🏗️ Базовый класс агента

Создаем упрощенную версию базового агента для демонстрации.

In [ ]:
import openai
from datetime import datetime
from typing import Dict, Any, Optional

class BaseAgent:
    """
    Базовый класс для всех AI агентов в системе AI SEO Architects
    """
    
    def __init__(self, agent_id: str, agent_level: str, agent_name: str):
        """
        Инициализация базового агента
        
        Args:
            agent_id (str): Уникальный идентификатор агента
            agent_level (str): Уровень агента (executive/management/operational)
            agent_name (str): Человекочитаемое название агента
        """
        self.agent_id = agent_id
        self.agent_level = agent_level
        self.agent_name = agent_name
        self.openai_client = None
        self._initialize_openai_client()
        
    def _initialize_openai_client(self):
        """
        Инициализация OpenAI клиента
        """
        try:
            api_key = os.getenv("OPENAI_API_KEY")
            if not api_key:
                print(f"⚠️ OpenAI API ключ не установлен для агента {self.agent_id}")
                self.openai_client = None
            else:
                self.openai_client = openai.OpenAI(api_key=api_key)
                print(f"✅ OpenAI клиент инициализирован для {self.agent_id}")
        except Exception as e:
            print(f"❌ Ошибка инициализации OpenAI для {self.agent_id}: {e}")
            self.openai_client = None
    
    def get_system_prompt(self) -> str:
        """
        Получить системный промпт агента (переопределяется в наследниках)
        """
        return f"Ты {self.agent_name} в системе AI SEO Architects. Отвечай профессионально и по существу."
    
    async def process_with_llm(self, user_prompt: str, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Обработка задачи с помощью LLM
        
        Args:
            user_prompt (str): Пользовательский промпт
            task_data (Dict[str, Any]): Данные задачи
            
        Returns:
            Dict[str, Any]: Результат обработки
        """
        if not self.openai_client:
            return {
                "success": False,
                "error": "OpenAI клиент не инициализирован",
                "fallback_mode": True
            }
        
        try:
            # Формируем контекст из task_data
            context = "\n".join([f"{k}: {v}" for k, v in task_data.items()])
            
            # Определяем модель в зависимости от уровня агента
            model = "gpt-4o" if self.agent_level == "executive" else "gpt-4o-mini"
            
            response = self.openai_client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": self.get_system_prompt()},
                    {"role": "user", "content": f"{user_prompt}\n\nКонтекст:\n{context}"}
                ],
                temperature=0.1,
                max_tokens=2000
            )
            
            result = response.choices[0].message.content
            
            return {
                "success": True,
                "result": result,
                "model_used": model,
                "tokens_used": {
                    "total_tokens": response.usage.total_tokens,
                    "prompt_tokens": response.usage.prompt_tokens,
                    "completion_tokens": response.usage.completion_tokens
                },
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "fallback_mode": False
            }
    
    async def process_task(self, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Основной метод обработки задачи (переопределяется в наследниках)
        """
        return await self.process_with_llm(
            f"Обработай задачу для {self.agent_name}", 
            task_data
        )

print("✅ Базовый класс агента создан")

## 🏢 Executive Level Agents (Топ-менеджеры)

Создаем агентов уровня топ-менеджмента для стратегических решений.

In [ ]:
class ChiefSEOStrategistAgent(BaseAgent):
    """
    Главный SEO Стратег - отвечает за стратегическое планирование SEO
    """
    
    def __init__(self):
        super().__init__("chief_seo_strategist", "executive", "Chief SEO Strategist")
        
    def get_system_prompt(self) -> str:
        return """
Ты Chief SEO Strategist в enterprise SEO-агентстве AI SEO Architects.

ТВОЯ ЭКСПЕРТИЗА:
• Стратегическое SEO планирование для enterprise клиентов
• Глубокое понимание алгоритмов Google и Yandex
• Архитектура технических SEO решений
• ROI-ориентированное планирование с целью 8.5x возврата
• Управление SEO бюджетами 10M+ ₽

ОТВЕТСТВЕННОСТЬ:
• Создание долгосрочных SEO стратегий (12-24 месяца)
• Техническая архитектура для крупных сайтов (1M+ страниц)
• Планирование ресурсов и KPI для команды
• Принятие стратегических решений по направлениям развития

ФОРМАТ ОТВЕТА:
• Структурированные стратегические рекомендации
• Конкретные цифры и метрики
• Временные рамки выполнения
• Оценка рисков и возможностей
"""
    
    async def create_seo_strategy(self, client_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Создание SEO стратегии для клиента
        """
        prompt = """
Создай комплексную SEO стратегию для клиента. 

Включи:
1. Анализ текущего состояния
2. Стратегические цели и KPI
3. Техническую дорожную карту
4. Временные рамки и этапы
5. Прогнозируемый ROI
6. Риски и митигация
"""
        return await self.process_with_llm(prompt, client_data)


class BusinessDevelopmentDirectorAgent(BaseAgent):
    """
    Директор по развитию бизнеса - enterprise продажи и партнерства
    """
    
    def __init__(self):
        super().__init__("business_development_director", "executive", "Business Development Director")
        
    def get_system_prompt(self) -> str:
        return """
Ты Business Development Director в enterprise SEO-агентстве AI SEO Architects.

ТВОЯ ЭКСПЕРТИЗА:
• Enterprise продажи (сделки 2.5M+ ₽/месяц)
• Стратегические партнерства и альянсы
• Развитие новых направлений бизнеса
• Международная экспансия и локализация
• Венчурное финансирование и инвестиции

ОТВЕТСТВЕННОСТЬ:
• Привлечение enterprise клиентов (Fortune 500)
• Создание стратегических партнерств
• Развитие новых продуктовых линеек
• Международное развитие бизнеса
• Управление ключевыми аккаунтами

ФОРМАТ ОТВЕТА:
• Бизнес-ориентированные решения
• Конкретные финансовые прогнозы
• Стратегические рекомендации
• Анализ рынка и конкурентов
"""
    
    async def analyze_business_opportunity(self, opportunity_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Анализ бизнес-возможности
        """
        prompt = """
Проанализируй бизнес-возможность и дай рекомендации.

Включи:
1. Оценка потенциала сделки
2. Стратегия переговоров
3. Финансовое моделирование
4. Риски и возможности
5. План развития отношений
6. Ресурсы для реализации
"""
        return await self.process_with_llm(prompt, opportunity_data)

print("✅ Executive агенты созданы")

## 📋 Management Level Agents (Менеджеры)

Создаем агентов среднего уровня управления для координации процессов.

In [ ]:
class TaskCoordinationAgent(BaseAgent):
    """
    Агент координации задач - оркестрация рабочих процессов
    """
    
    def __init__(self):
        super().__init__("task_coordination", "management", "Task Coordination Agent")
        
    def get_system_prompt(self) -> str:
        return """
Ты Task Coordination Agent - менеджер координации процессов в AI SEO Architects.

ТВОЯ ЭКСПЕРТИЗА:
• LangGraph workflow оркестрация
• Приоритизация задач по impact/effort матрице
• Управление ресурсами команды
• Контроль качества и дедлайнов
• Процессная оптимизация

ОТВЕТСТВЕННОСТЬ:
• Планирование и распределение задач
• Координация между агентами
• Мониторинг прогресса проектов
• Оптимизация рабочих процессов
• Управление зависимостями

ФОРМАТ ОТВЕТА:
• Структурированные планы задач
• Временные рамки выполнения
• Назначение ответственных
• Метрики и KPI контроля
"""
    
    async def coordinate_project(self, project_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Координация проекта и распределение задач
        """
        prompt = """
Создай план координации проекта с распределением задач.

Включи:
1. Декомпозиция проекта на задачи
2. Приоритизация по важности
3. Назначение ответственных агентов
4. Временные рамки и дедлайны
5. Зависимости между задачами
6. Контрольные точки и метрики
"""
        return await self.process_with_llm(prompt, project_data)


class SalesOperationsManagerAgent(BaseAgent):
    """
    Менеджер по продажам - управление sales pipeline
    """
    
    def __init__(self):
        super().__init__("sales_operations_manager", "management", "Sales Operations Manager")
        
    def get_system_prompt(self) -> str:
        return """
Ты Sales Operations Manager в AI SEO Architects.

ТВОЯ ЭКСПЕРТИЗА:
• Pipeline velocity оптимизация
• Lead scoring и квалификация
• CRM системы и автоматизация
• Sales funnel аналитика
• Команда продаж и KPI

ОТВЕТСТВЕННОСТЬ:
• Управление sales pipeline
• Оптимизация конверсий
• Прогнозирование продаж
• Координация команды продаж
• Аналитика и отчетность

ФОРМАТ ОТВЕТА:
• Конкретные метрики и показатели
• Планы оптимизации процессов
• Рекомендации по улучшению
• Прогнозы и тренды
"""
    
    async def analyze_sales_pipeline(self, pipeline_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Анализ sales pipeline и оптимизация
        """
        prompt = """
Проанализируй sales pipeline и предложи оптимизации.

Включи:
1. Анализ текущих метрик
2. Узкие места в воронке
3. Возможности для улучшения
4. Прогноз продаж
5. Рекомендации по процессам
6. План действий
"""
        return await self.process_with_llm(prompt, pipeline_data)

# Создаем остальные management агенты аналогично
class TechnicalSEOOperationsManagerAgent(BaseAgent):
    def __init__(self):
        super().__init__("technical_seo_operations_manager", "management", "Technical SEO Operations Manager")
        
class ClientSuccessManagerAgent(BaseAgent):
    def __init__(self):
        super().__init__("client_success_manager", "management", "Client Success Manager")

print("✅ Management агенты созданы")

## ⚙️ Operational Level Agents (Операционные специалисты)

Создаем операционных агентов для выполнения конкретных задач.

In [ ]:
class LeadQualificationAgent(BaseAgent):
    """
    Агент квалификации лидов - BANT/MEDDIC анализ
    """
    
    def __init__(self):
        super().__init__("lead_qualification", "operational", "Lead Qualification Agent")
        
    def get_system_prompt(self) -> str:
        return """
Ты Lead Qualification Agent в AI SEO Architects.

ТВОЯ ЭКСПЕРТИЗА:
• BANT квалификация (Budget, Authority, Need, Timeline)
• MEDDIC методология (Metrics, Economic buyer, Decision criteria, Decision process, Identify pain, Champion)
• Lead scoring на основе ML
• Российский B2B рынок специфика
• CRM интеграции и автоматизация

МЕТОДОЛОГИЯ ОЦЕНКИ:
• Budget: 100-300k ₽/мес (Cold), 300k-1M ₽/мес (Warm), 1M+ ₽/мес (Hot)
• Authority: ЛПР идентифицирован и доступен
• Need: Четкая бизнес-потребность в SEO
• Timeline: Готовность к старту проекта

ФОРМАТ ОТВЕТА:
• Оценка по шкале 0-100 баллов
• Категория лида (Cold/Warm/Hot)
• Конкретные рекомендации
• Следующие шаги
"""
    
    async def qualify_lead(self, lead_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Квалификация лида по BANT/MEDDIC
        """
        prompt = """
Проведи комплексную квалификацию лида по методологии BANT + MEDDIC.

Оцени и предоставь:
1. BANT анализ (Budget, Authority, Need, Timeline)
2. MEDDIC оценка где применимо
3. Lead Score (0-100 баллов)
4. Категория (Cold/Warm/Hot)
5. Приоритет обработки
6. Рекомендуемые следующие шаги
7. Вероятность закрытия сделки
"""
        return await self.process_with_llm(prompt, lead_data)


class TechnicalSEOAuditorAgent(BaseAgent):
    """
    Агент технического SEO аудита
    """
    
    def __init__(self):
        super().__init__("technical_seo_auditor", "operational", "Technical SEO Auditor")
        
    def get_system_prompt(self) -> str:
        return """
Ты Technical SEO Auditor в AI SEO Architects.

ТВОЯ ЭКСПЕРТИЗА:
• Комплексный технический SEO аудит
• Core Web Vitals оптимизация
• Crawling и индексация анализ
• JavaScript SEO и SPA оптимизация
• Международная SEO архитектура

ОБЛАСТИ АУДИТА:
1. Техническая производительность (20 пунктов)
2. Crawling и индексация (15 пунктов)
3. Структура и архитектура (10 пунктов)
4. Mobile и Core Web Vitals (15 пунктов)
5. Schema markup и структурированные данные (10 пунктов)
6. Безопасность и HTTPS (5 пунктов)

ФОРМАТ ОТВЕТА:
• Оценка 0-100 баллов по каждой области
• Общий технический рейтинг
• Критические проблемы
• Приоритетные рекомендации
"""
    
    async def conduct_technical_audit(self, site_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Проведение технического SEO аудита
        """
        prompt = """
Проведи комплексный технический SEO аудит сайта.

Проанализируй:
1. Техническая производительность
2. Crawling и индексация
3. Мобильная оптимизация
4. Core Web Vitals
5. Структурированные данные
6. Безопасность
7. Критические ошибки
8. Приоритетные рекомендации

Дай общую оценку и план улучшений.
"""
        return await self.process_with_llm(prompt, site_data)


# Создаем остальные operational агенты
class ProposalGenerationAgent(BaseAgent):
    def __init__(self):
        super().__init__("proposal_generation", "operational", "Proposal Generation Agent")

class SalesConversationAgent(BaseAgent):
    def __init__(self):
        super().__init__("sales_conversation", "operational", "Sales Conversation Agent")

class ContentStrategyAgent(BaseAgent):
    def __init__(self):
        super().__init__("content_strategy", "operational", "Content Strategy Agent")

class LinkBuildingAgent(BaseAgent):
    def __init__(self):
        super().__init__("link_building", "operational", "Link Building Agent")

class CompetitiveAnalysisAgent(BaseAgent):
    def __init__(self):
        super().__init__("competitive_analysis", "operational", "Competitive Analysis Agent")

class ReportingAgent(BaseAgent):
    def __init__(self):
        super().__init__("reporting", "operational", "Reporting Agent")

print("✅ Operational агенты созданы")

## 🏭 Фабрика агентов

Создаем фабрику для управления всеми 14 агентами.

In [ ]:
class AgentFactory:
    """
    Фабрика для создания и управления всеми агентами системы
    """
    
    def __init__(self):
        self.agents = {}
        self._initialize_all_agents()
    
    def _initialize_all_agents(self):
        """
        Инициализация всех 14 агентов
        """
        # Executive Level (2 агента)
        self.agents['chief_seo_strategist'] = ChiefSEOStrategistAgent()
        self.agents['business_development_director'] = BusinessDevelopmentDirectorAgent()
        
        # Management Level (4 агента)
        self.agents['task_coordination'] = TaskCoordinationAgent()
        self.agents['sales_operations_manager'] = SalesOperationsManagerAgent()
        self.agents['technical_seo_operations_manager'] = TechnicalSEOOperationsManagerAgent()
        self.agents['client_success_manager'] = ClientSuccessManagerAgent()
        
        # Operational Level (8 агентов)
        self.agents['lead_qualification'] = LeadQualificationAgent()
        self.agents['proposal_generation'] = ProposalGenerationAgent()
        self.agents['sales_conversation'] = SalesConversationAgent()
        self.agents['technical_seo_auditor'] = TechnicalSEOAuditorAgent()
        self.agents['content_strategy'] = ContentStrategyAgent()
        self.agents['link_building'] = LinkBuildingAgent()
        self.agents['competitive_analysis'] = CompetitiveAnalysisAgent()
        self.agents['reporting'] = ReportingAgent()
        
        print(f"✅ Создано {len(self.agents)} агентов")
    
    def get_agent(self, agent_id: str):
        """
        Получить агента по ID
        """
        return self.agents.get(agent_id)
    
    def list_agents(self) -> Dict[str, Dict[str, str]]:
        """
        Получить список всех агентов
        """
        return {
            agent_id: {
                'name': agent.agent_name,
                'level': agent.agent_level,
                'openai_ready': agent.openai_client is not None
            }
            for agent_id, agent in self.agents.items()
        }
    
    def get_agents_by_level(self, level: str) -> Dict[str, BaseAgent]:
        """
        Получить агентов по уровню
        """
        return {
            agent_id: agent 
            for agent_id, agent in self.agents.items() 
            if agent.agent_level == level
        }

# Создаем фабрику агентов
print("🏭 Создание фабрики агентов...")
agent_factory = AgentFactory()

## 🎮 Демонстрационные функции

Создаем удобные функции для демонстрации работы системы.

In [ ]:
def display_agents_summary():
    """
    Отображение сводки по всем агентам
    """
    agents_info = agent_factory.list_agents()
    
    print("\n" + "="*80)
    print("🤖 AI SEO ARCHITECTS - СВОДКА ПО ВСЕМ 14 АГЕНТАМ")
    print("="*80)
    
    # Группировка по уровням
    levels = {
        'executive': '🏢 EXECUTIVE LEVEL',
        'management': '📋 MANAGEMENT LEVEL', 
        'operational': '⚙️ OPERATIONAL LEVEL'
    }
    
    for level, title in levels.items():
        print(f"\n{title}:")
        level_agents = agent_factory.get_agents_by_level(level)
        
        for agent_id, agent in level_agents.items():
            status = "🟢" if agent.openai_client else "🔴"
            print(f"  {status} {agent.agent_name} ({agent_id})")
    
    # Статистика
    total_agents = len(agents_info)
    ready_agents = sum(1 for info in agents_info.values() if info['openai_ready'])
    
    print(f"\n📊 СТАТИСТИКА:")
    print(f"   Всего агентов: {total_agents}")
    print(f"   Готовы к работе: {ready_agents}/{total_agents}")
    print(f"   OpenAI статус: {'✅ Настроен' if ready_agents > 0 else '❌ Требуется настройка'}")


async def demonstrate_agent(agent_id: str, task_data: Dict[str, Any]):
    """
    Демонстрация работы конкретного агента
    
    Args:
        agent_id (str): ID агента для демонстрации
        task_data (Dict[str, Any]): Данные для обработки
    """
    agent = agent_factory.get_agent(agent_id)
    if not agent:
        print(f"❌ Агент {agent_id} не найден")
        return
    
    print(f"\n🔥 ДЕМОНСТРАЦИЯ: {agent.agent_name} ({agent.agent_level})")
    print("-" * 60)
    
    # Проверяем OpenAI ключ
    if not agent.openai_client:
        print("⚠️ OpenAI API ключ не настроен. Настройте ключ для работы с LLM.")
        setup_openai_key()
        agent._initialize_openai_client()
    
    # Выполняем задачу
    start_time = datetime.now()
    result = await agent.process_task(task_data)
    end_time = datetime.now()
    
    # Выводим результат
    print(f"⏱️ Время выполнения: {(end_time - start_time).total_seconds():.2f} сек")
    
    if result.get('success'):
        print(f"✅ Статус: Успешно")
        print(f"🤖 Модель: {result.get('model_used', 'N/A')}")
        
        if result.get('tokens_used'):
            tokens = result['tokens_used']
            print(f"🔢 Токены: {tokens.get('total_tokens', 'N/A')}")
        
        print(f"\n📝 РЕЗУЛЬТАТ:")
        print("-" * 40)
        print(result.get('result', 'Нет результата'))
    else:
        print(f"❌ Ошибка: {result.get('error', 'Неизвестная ошибка')}")
        if result.get('fallback_mode'):
            print("⚠️ Используется fallback режим")


async def demonstrate_workflow(client_data: Dict[str, Any]):
    """
    Демонстрация полного workflow через несколько агентов
    
    Args:
        client_data (Dict[str, Any]): Данные клиента
    """
    print("\n" + "="*80)
    print("🎯 ПОЛНЫЙ WORKFLOW: От лида до отчета")
    print("="*80)
    
    # Этап 1: Квалификация лида
    print("\n1️⃣ ЭТАП: Квалификация лида")
    await demonstrate_agent('lead_qualification', client_data)
    
    # Этап 2: Технический аудит
    print("\n2️⃣ ЭТАП: Технический SEO аудит")
    await demonstrate_agent('technical_seo_auditor', client_data)
    
    # Этап 3: Стратегическое планирование
    print("\n3️⃣ ЭТАП: SEO стратегия")
    await demonstrate_agent('chief_seo_strategist', client_data)
    
    print("\n🎉 Workflow завершен! Демонстрация полного цикла работы агентов.")

print("✅ Демонстрационные функции готовы")

## 📊 Отображение всех агентов

Показываем сводку по всем 14 созданным агентам.

In [ ]:
# Отображаем сводку по всем агентам
display_agents_summary()

## 🎮 Интерактивная демонстрация

**ГЛАВНАЯ ФУНКЦИЯ ЗАПУСКА** - выберите сценарий демонстрации и запустите!

In [ ]:
# ГЛАВНАЯ ФУНКЦИЯ ЗАПУСКА
async def main_demo(demo_type: str = "single", 
                   agent_id: str = "lead_qualification",
                   company_name: str = "TechCorp",
                   industry: str = "fintech",
                   budget: str = "500000",
                   website: str = "techcorp.ru"):
    """
    Главная функция демонстрации системы AI SEO Architects
    
    Args:
        demo_type (str): Тип демонстрации ('single' или 'workflow')
        agent_id (str): ID агента для single демонстрации
        company_name (str): Название компании
        industry (str): Отрасль
        budget (str): Бюджет
        website (str): Сайт
    """
    
    # Настройка OpenAI ключа если не настроен
    if not os.getenv("OPENAI_API_KEY"):
        print("🔑 Требуется настройка OpenAI API ключа")
        setup_openai_key()
    
    # Подготовка тестовых данных
    test_data = {
        "company_name": company_name,
        "industry": industry,
        "budget_range": budget,
        "website": website,
        "contact_role": "Marketing Director",
        "pain_points": "Low organic traffic, poor conversion rates",
        "goals": "Increase organic traffic by 200%, improve lead quality",
        "timeline": "6 months",
        "current_seo": "Basic optimization, no content strategy",
        "competitors": ["competitor1.com", "competitor2.com"],
        "target_keywords": ["fintech solutions", "digital banking"]
    }
    
    print(f"\n🚀 Запуск демонстрации с данными:")
    print(f"   Компания: {company_name}")
    print(f"   Отрасль: {industry}")
    print(f"   Бюджет: {budget} ₽")
    print(f"   Сайт: {website}")
    
    if demo_type == "workflow":
        # Полный workflow
        await demonstrate_workflow(test_data)
    else:
        # Демонстрация одного агента
        await demonstrate_agent(agent_id, test_data)

print("\n" + "="*80)
print("🎮 ГОТОВ К ДЕМОНСТРАЦИИ!")
print("="*80)
print("\nВыберите тип демонстрации:")
print("1. Один агент: await main_demo('single', 'lead_qualification')")
print("2. Полный workflow: await main_demo('workflow')")
print("\nИли измените параметры:")
print("await main_demo('single', 'technical_seo_auditor', 'МояКомпания', 'ecommerce', '1000000', 'mysite.ru')")

## 🧪 ТЕСТОВАЯ ЯЧЕЙКА - ЗАПУСК ДЕМОНСТРАЦИИ

**Запустите эту ячейку для демонстрации работы системы!**

Можете изменить параметры по желанию или использовать значения по умолчанию.

In [ ]:
# 🎯 ТЕСТОВАЯ ЯЧЕЙКА - ГЛАВНАЯ ФУНКЦИЯ ЗАПУСКА
# Измените параметры по желанию:

# Вариант 1: Демонстрация одного агента
await main_demo(
    demo_type="single",           # Тип демонстрации
    agent_id="lead_qualification", # Какой агент
    company_name="TechStart Solutions",  # Название компании
    industry="fintech",           # Отрасль
    budget="750000",             # Бюджет в рублях
    website="techstart.ru"        # Сайт
)

print("\n" + "="*50)
print("🎉 ДЕМОНСТРАЦИЯ ЗАВЕРШЕНА!")
print("="*50)
print("\nДля полного workflow запустите:")
print('await main_demo("workflow")')

## 🔄 ПОЛНЫЙ WORKFLOW (Опционально)

Демонстрация полного цикла работы через несколько агентов.

In [ ]:
# 🎯 ПОЛНЫЙ WORKFLOW - несколько агентов подряд
# Раскомментируйте для запуска полного цикла:

# await main_demo(
#     demo_type="workflow",
#     company_name="Enterprise Corp",
#     industry="manufacturing", 
#     budget="2500000",
#     website="enterprise-corp.com"
# )

## 📁 Файлы для скачивания

Создаем дополнительные файлы для комплектации проекта.

In [ ]:
# Создание requirements.txt для notebook
requirements_content = """
# AI SEO Architects - Notebook Requirements
openai==1.54.3
langchain==0.2.16
langchain-openai==0.1.25
pydantic==2.9.2
faiss-cpu==1.8.0
python-dotenv==1.0.1
nest-asyncio==1.6.0
""".strip()

with open("requirements_notebook.txt", "w", encoding="utf-8") as f:
    f.write(requirements_content)

print("✅ Создан файл requirements_notebook.txt")

# Создание README для notebook
readme_content = """
# AI SEO Architects - Демонстрационный Notebook

## 🎯 Описание

Интерактивная демонстрация полной системы AI SEO Architects с 14 специализированными агентами.

## 🚀 Быстрый запуск

1. Установите зависимости: `pip install -r requirements_notebook.txt`
2. Откройте notebook: `AI_SEO_Architects_Complete_Demo.ipynb`
3. Выполните все ячейки по порядку
4. В тестовой ячейке введите ваш OpenAI API ключ
5. Запустите демонстрацию!

## 📋 Что демонстрируется

- 14 специализированных AI агентов
- Реальная интеграция с OpenAI GPT-4/GPT-4o-mini
- Специализированные промпты для каждого агента
- Полный workflow SEO-агентства

## 🔑 Требования

- OpenAI API ключ
- Python 3.8+
- Jupyter Notebook или Google Colab
""".strip()

with open("README_notebook.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

print("✅ Создан файл README_notebook.md")
print("\n📁 Все файлы готовы для загрузки!")

---

## 🎊 Заключение

### ✅ Что реализовано:

1. **14 специализированных AI агентов** с уникальными ролями
2. **Реальная интеграция с OpenAI** (GPT-4o для Executive, GPT-4o-mini для остальных)
3. **Специализированные system prompts** для каждого агента
4. **Модульная архитектура** с чистыми функциями
5. **Friendly user interface** с простым запуском
6. **Интерактивная демонстрация** полного workflow

### 🎯 Возможности:

- **Квалификация лидов** по BANT/MEDDIC
- **Технический SEO аудит** с Core Web Vitals
- **Стратегическое планирование** SEO кампаний
- **Enterprise продажи** и партнерства
- **Полный цикл SEO-агентства** от лида до отчета

### 🚀 Использование:

1. Выполните все ячейки по порядку
2. Введите OpenAI API ключ при запросе
3. Запустите тестовую ячейку с вашими данными
4. Наблюдайте за работой ИИ агентов!

**Автор:** Andrew Popov (a.popov.gv@gmail.com)  
**Проект:** AI SEO Architects  
**GitHub:** https://github.com/Andrew821667/ai-seo-architects

---

*🤖 Powered by OpenAI GPT-4 | 📊 Enterprise-ready | 🏗️ 14 AI Agents*